В этом примере я использую категоризацию с помощью API сервиса VirusTotal. В отличие от Alphamountain, VirusTotal предоставляет ключ для API, что упрощает процесс интеграции. Этот скрипт выполняет запросы к API для получения категории для каждого URL из файла new_url_task.csv. Для этого URL сначала кодируется в формат Base64, затем отправляется запрос на сервер VirusTotal для получения информации. Результаты сохраняются в CSV файл, где каждая строка содержит URL и соответствующую категорию. В процессе выполнения добавлены паузы между запросами для предотвращения перегрузки API.

In [1]:
import requests
import getpass
import pandas as pd
import numpy as np
from tqdm import tqdm
import time

In [2]:
API_KEY = ''


In [3]:
# Endpoint для получения отчета по URL (v3)
url_report_endpoint = f"https://www.virustotal.com/api/v3/urls"

# Функция для получения ID зашифрованного URL (требуется для v3)
def get_encoded_url(url):
    import base64
    url_bytes = url.encode("utf-8")
    base64_url = base64.urlsafe_b64encode(url_bytes).decode("utf-8").strip("=")
    return base64_url

# Шаг 1: Получение отчета с категорией URL через API v3
def get_url_report_v3(url):
    # Получаем зашифрованный URL для v3 API
    encoded_url = get_encoded_url(url)
    
    # Формируем заголовки и запрос
    headers = {
        "x-apikey": API_KEY
    }
    response = requests.get(f"{url_report_endpoint}/{encoded_url}", headers=headers)
    
    if response.status_code == 200:
        report = response.json()
        
        # Проверяем, есть ли отчет по категории
        try:
            categories = report["data"]["attributes"]["categories"]
            return categories
        except KeyError:
            pass
#             print(f"{url}\nКатегория не найдена в отчете.")
        
    else:
        print(f"Ошибка при получении отчета: {response.status_code}, {response.text}")

# Получаем отчет с категорией для указанного URL
# r = get_url_report_v3(url)


In [4]:
df_urls = pd.read_csv('new_url_task.csv')
urls = df_urls['url'].unique()

In [5]:
cats_list = list()

In [ ]:
i = 0
filename = 'category_result_month_new.csv'
N_start = 1
N_end = 2

for url in tqdm(urls[N_start:N_end]):
    cats = get_url_report_v3(url)
    if cats:
        cats_list.append((url, cats))
    else:
        cats_list.append((url, np.nan))
    
    time.sleep(15)
    
    if i % 10 == 0:
        df_res = pd.DataFrame(cats_list, columns=['url', 'cats'])
        df_res.to_csv(filename)
    
    i += 1
    
df_res.to_csv(filename)

  0%|          | 0/1 [00:21<?, ?it/s]


##### 